In [1]:
import logging
import pandas as pd
import os

#log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 25 # ANALYSIS_LEVEL
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

from dotenv import load_dotenv
load_dotenv()
secret_name = "OPENAI_API_KEY_CEMAD"
openai_api_key = os.getenv(secret_name)
key = os.getenv('DECRYPTION_KEY_CEMAD')


In [3]:

from regulations_rag.embeddings import  EmbeddingParameters
from regulations_rag.rerank import RerankAlgos
from regulations_rag.corpus_chat import ChatParameters
from regulations_rag.path_search import PathSearch
from regulations_rag.path_rag import PathRAG
from regulations_rag.path_no_rag_data import PathNoRAGData

import importlib
import cemad_rag.cemad_corpus_index
importlib.reload(cemad_rag.cemad_corpus_index)
from cemad_rag.cemad_corpus_index import CEMADCorpusIndex

import cemad_rag.corpus_chat_cemad
importlib.reload(cemad_rag.corpus_chat_cemad)
from cemad_rag.corpus_chat_cemad import CorpusChatCEMAD

import cemad_rag.path_suggest_alternatives
importlib.reload(cemad_rag.path_suggest_alternatives)
from cemad_rag.path_suggest_alternatives import PathSuggestAlternatives

from IPython.display import Markdown, display


In [4]:
chat_parameters = ChatParameters(chat_model = "gpt-4o", api_key = openai_api_key, temperature = 0, max_tokens = 4096, token_limit_when_truncating_message_queue = 3500)

embedding_parameters = EmbeddingParameters("text-embedding-3-large", 1024)
corpus_index = CEMADCorpusIndex(key)

rerank_algo  = RerankAlgos.LLM
if rerank_algo == RerankAlgos.LLM:
    rerank_algo.params["openai_client"] = chat_parameters.openai_client
    rerank_algo.params["model_to_use"] = chat_parameters.model
    rerank_algo.params["user_type"] = corpus_index.user_type
    rerank_algo.params["corpus_description"] = corpus_index.corpus_description
    rerank_algo.params["final_token_cap"] = 5000 # can go large with the new models


chat = CorpusChatCEMAD(embedding_parameters = embedding_parameters, chat_parameters = chat_parameters, corpus_index = corpus_index, rerank_algo = RerankAlgos.LLM, user_name_for_logging = 'test_user')
chat.strict_rag = False
path_suggest_alternatives = PathSuggestAlternatives(chat_parameters = chat_parameters, corpus_index = corpus_index, embedding_parameters = embedding_parameters, rerank_algo = rerank_algo)
path_search = PathSearch(corpus_index = corpus_index, chat_parameters = chat_parameters, embedding_parameters = embedding_parameters, rerank_algo = rerank_algo)
path_rag = PathRAG(corpus_index = corpus_index, chat_parameters = chat_parameters)
path_no_rag_data = PathNoRAGData(corpus_index = corpus_index, chat_parameters = chat_parameters)


Usage
```
user_content = "Can I buy property overseas using my credit card?"
result = path_suggest_alternatives.suggest_alternative_questions(message_history = [], current_user_message = {'role': 'user', 'content': user_content})

result = path_no_rag_data.query_no_rag_data(message_history = [], current_user_message = {'role': 'user', 'content': user_content})

workflow_triggered, relevant_definitions, relevant_sections = path_search.similarity_search(user_question = user_content)
result = path_rag.perform_RAG_path(message_history = [], current_user_message = {'role': 'user', 'content': user_content, 'reference_material': {'definitions': relevant_definitions, 'sections': relevant_sections}})
```

In [7]:
user_content = "Can I buy property overseas using my credit card?"
result = path_suggest_alternatives.suggest_alternative_questions(message_history = [], current_user_message = {'role': 'user', 'content': user_content})
result


ANALYSIS:cemad_rag.path_suggest_alternatives:Suggesting alternative questions


{'role': 'assistant',
 'content': 'The system has suggested the following alternative questions: \n\nCan I purchase property in a foreign country using a credit card?',
 'assistant_response': AlternativeQuestionResponse(alternatives=['Can I purchase property in a foreign country using a credit card?'])}

In [17]:
user_content = "Can I purchase property in a foreign country using a credit card?"
workflow_triggered, relevant_definitions, relevant_sections = path_search.similarity_search(user_question = user_content)

# result = path_rag.perform_RAG_path(message_history = [], current_user_message = {'role': 'user', 'content': user_content, 'reference_material': {'definitions': relevant_definitions, 'sections': relevant_sections}})
# result


In [19]:
relevant_sections

,section_reference,text,source,embedding,document,cosine_distance,regulation_text,token_count
0,B.2(B)(i),How can residents finance foreign asset purcha...,question,"[-0.05637288838624954, -0.0065882159397006035,...",CEMAD,0.374013,B.2 Capital transfers\n (B) Private individ...,2444


In [18]:
print(len(relevant_sections))
print(relevant_sections.iloc[0]['text'])
print(relevant_sections.iloc[0]['regulation_text'])


1
How can residents finance foreign asset purchases?
B.2 Capital transfers
    (B) Private individuals resident in South Africa
        (i) Foreign investments by private individuals (natural persons) resident in South Africa
            (a) Authorised Dealers may allow the transfer, as a foreign capital allowance, of up to a total amount of R10 million per calendar year per private individual who is a taxpayer in good standing and is 18 years and older, for investment purposes abroad. The funds to be transferred must be converted to foreign currency by the Authorised Dealer and may also be held in a resident foreign currency account in the name of the resident with any Authorised Dealer.
            (b) Authorised Dealers are advised that a valid green bar-coded South African identity document or a Smart identity document card is the only acceptable document proving residency in South Africa.
            (c) Prior to authorising the transaction, Authorised Dealers must ensure that the